<a href="https://colab.research.google.com/github/asgarali429/Machine-learning/blob/main/Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
df= pd.read_csv('covid_toy.csv')
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [3]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [4]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [5]:
from sklearn.model_selection import train_test_split

xtrain,xtest,ytrain,ytest = train_test_split(df.iloc[:,:-1],df.iloc[:,-1],test_size=0.2,random_state=42)

Encoding using sklearn

In [6]:
## Handling missing value
si = SimpleImputer()
xtrain_fever = si.fit_transform(xtrain[['fever']])
xtest_fever = si.transform(xtest[['fever']])
xtrain_fever.shape

(80, 1)

In [7]:
# ordinalEncdoing
oe = OrdinalEncoder(categories=[['Mild','Strong']])
xtrain_cough = oe.fit_transform(xtrain[['cough']])
xtest_cough = oe.transform(xtest[['cough']])
xtrain_cough.shape

(80, 1)

In [8]:
# OneHotEncoding
ohe = OneHotEncoder(drop='first',sparse_output=False)
xtrain_gender_city = ohe.fit_transform(xtrain[['gender','city']])
xtest_gender_city = ohe.fit_transform(xtest[['gender','city']])

In [9]:
#Extracting Age
xtrain_age = xtrain.drop(columns=['fever','cough','gender','city']).values
xtest_age = xtest.drop(columns=['fever','cough','gender','city']).values


In [10]:
xtrain_transfrom=np.concatenate((xtrain_age,xtrain_gender_city,xtrain_fever,xtrain_cough),axis=1)
xtest_transfrom=np.concatenate((xtest_age,xtest_gender_city,xtest_fever,xtest_cough),axis=1)


##Encoding using ColumnTransformer
which is whey easier to compute as compare to compute every column and then fit tranform

In [11]:
from sklearn.compose import ColumnTransformer
transformer = ColumnTransformer(transformers=[
    ('tnfi',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(drop='first',sparse_output=False),['gender','city'])
],remainder='passthrough')

In [12]:
transformer.fit_transform(xtrain).shape

(80, 7)

In [13]:
transformer.transform(xtest).shape

(20, 7)

###Pipeline

In [15]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline

In [57]:
df = pd.read_csv('train.csv')

In [58]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [59]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)

In [60]:
xtrain,xtest,ytrain,ytest = train_test_split(df.drop(columns=['Survived']),df['Survived'],test_size=0.2,random_state=42)

In [61]:
xtrain.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S


In [62]:
ytrain.sample(5)

589    0
111    0
430    1
649    1
441    0
Name: Survived, dtype: int64

In [63]:
trf1 = ColumnTransformer([
    ('impute_Age',SimpleImputer(),[2]),
    ('ipute_Embarked',SimpleImputer(strategy='most_frequent'),[6])
],remainder='passthrough')

In [64]:
trf2 = ColumnTransformer([
    ('ohe_sex_embarked',OneHotEncoder(drop='first',sparse_output=False,handle_unknown='ignore'),[1,6])
],remainder='passthrough')

In [80]:
trf3 = ColumnTransformer([
    ('scale',MinMaxScaler(),[0,8])
])

In [81]:
trf4 = DecisionTreeClassifier()

Create a pipeline

In [82]:
pipe = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf4',trf4)
])

In [83]:
pipe.fit(xtrain,ytrain)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_Age', SimpleImputer(),
                                                  [2]),
                                                 ('ipute_Embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  [0, 8])])),
                ('trf4', DecisionTreeClassifier())])

##Explore the pipeline

In [84]:
pipe.named_steps

{'trf1': ColumnTransformer(remainder='passthrough',
                   transformers=[('impute_Age', SimpleImputer(), [2]),
                                 ('ipute_Embarked',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'trf2': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_sex_embarked',
                                  OneHotEncoder(drop='first',
                                                handle_unknown='ignore',
                                                sparse_output=False),
                                  [1, 6])]),
 'trf3': ColumnTransformer(transformers=[('scale', MinMaxScaler(), [0, 8])]),
 'trf4': DecisionTreeClassifier()}

In [ ]:
ypred = pipe.predict(xtest)

In [86]:

from sklearn.metrics import accuracy_score
accuracy_score(ytest,ypred)

0.5865921787709497

# cross validation using cross_val_score

In [ ]:

# cross validation using cross_val_score
from sklearn.model_selection import cross_val_score
cross_val_score(pipe, xtrain, ytrain, cv=5, scoring='accuracy').mean()

# gridsearchcv

In [88]:
# gridsearchcv
params = {
    'trf4__max_depth':[1,2,3,4,5,None]
}

In [ ]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(pipe, params, cv=5, scoring='accuracy')
grid.fit(xtrain, ytrain)

In [91]:
grid.best_score_

0.6235989362749926

In [92]:
grid.best_params_

{'trf4__max_depth': 1}

#exporting the pipeline

In [93]:
#exporting the pipeline
import pickle
pickle.dump(pipe,open('pipe.pkl','wb'))

In [95]:
pipe = pickle.load(open('pipe.pkl','rb'))

In [96]:
test_input2 = np.array([2, 'male', 31.0, 0, 0, 10.5, 'S'],dtype=object).reshape(1,7)

In [97]:
pipe.predict(test_input2)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(


array([0])